<a href="https://colab.research.google.com/github/RohitPatilRRP/valueLabs/blob/master/valueLabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import nltk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [107]:
url_train = 'https://raw.githubusercontent.com/RohitPatilRRP/valueLabs/master/Train.csv'
train = pd.read_csv(url_train)
train.columns = [x.lower() for x in train.columns]
train.head()

,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini..."
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T..."
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .'


In [108]:
url_test = 'https://raw.githubusercontent.com/RohitPatilRRP/valueLabs/master/Test.csv'
test = pd.read_csv(url_test)
test.columns = [x.lower() for x in test.columns]
test.head()

,question,answer_text
0,What 'S the main idea of the text ?,The lack of career -- based courses in US high...
1,"In the summer high season , Finland does nt se...",the sun is out at night
2,If you want to apply for Chinese Business Inte...,have to get confirmed at least twice
3,"That afternoon , the boy 's clothes were dry b...",nobody made room for him in the water .
4,Which of the following statements is NOT true ?,There are twelve countries in the World Wildli...


In [0]:
x_train = train.iloc[:,0:2]
y_train  = train.iloc[:,2:3]
x_test = train.iloc[:,0:2] 
y_test  = train.iloc[:,2:3]

In [0]:
import re
import csv

def createY(y):
  y['op0'] = np.nan
  y['op1'] = np.nan
  y['op2'] = np.nan
  y['op3'] = np.nan
  for i in range(0,len(y['distractor'])):
    cStr = y['distractor'].iloc[i]
    newStr = ['{}'.format(x) for x in list(csv.reader([cStr], delimiter=',', quotechar='"'))[0]]
    y['op0'].iloc[i] = newStr[0] if len(newStr) >= 1 else ""
    y['op1'].iloc[i] = newStr[1] if len(newStr) >= 2 else ""
    y['op2'].iloc[i] = newStr[2] if len(newStr) >= 3 else ""
    y['op3'].iloc[i] = newStr[3] if len(newStr) >= 4 else ""
  return y

In [115]:
y_train = createY(y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [116]:
y_train.head(3)

,distractor,op0,op1,op2,op3
0,"'outside the room at 3:00 p. m.', 'in the dini...",'outside the room at 3:00 p. m.','in the dining - room at 6:00 p. m.','in the dining - room from 7:30 a. m. to 9:15...,
1,"'If some tragedies occur again ', ' relevant d...",'If some tragedies occur again ',' relevant departments of the State Council s...,'Currently ',' the central government has established soun...
2,"'blame Tommy for his failing grades', 'blame T...",'blame Tommy for his failing grades','blame Tommy for his failing grades',,


In [117]:
y_test = createY(y_test)
y_test.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,distractor,op0,op1,op2,op3
0,"'outside the room at 3:00 p. m.', 'in the dini...",'outside the room at 3:00 p. m.','in the dining - room at 6:00 p. m.','in the dining - room from 7:30 a. m. to 9:15...,
1,"'If some tragedies occur again ', ' relevant d...",'If some tragedies occur again ',' relevant departments of the State Council s...,'Currently ',' the central government has established soun...
2,"'blame Tommy for his failing grades', 'blame T...",'blame Tommy for his failing grades','blame Tommy for his failing grades',,


In [118]:
#final training data
data_train = pd.concat([x_train,y_train],axis=1)
data_test = pd.concat([x_test,y_test],axis=1)
data_train.head(3)

,question,answer_text,distractor,op0,op1,op2,op3
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini...",'outside the room at 3:00 p. m.','in the dining - room at 6:00 p. m.','in the dining - room from 7:30 a. m. to 9:15...,
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d...",'If some tragedies occur again ',' relevant departments of the State Council s...,'Currently ',' the central government has established soun...
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T...",'blame Tommy for his failing grades','blame Tommy for his failing grades',,


In [0]:
def dataPreProc(data):
  del data['distractor']
  del data['op3']
  data.replace('', np.nan, inplace=True)
  data.dropna(inplace=True)
  for c in data:
    data[c] = data[c].apply(lambda x:' '.join(x.lower() for x in x.split()))
    data[c]= data[c].apply(lambda x: ' '.join(x for x in x.split() if x not in string.punctuation))
    data[c]= data[c].str.replace('[^\w\s]','')
    data[c]= data[c].apply(lambda x: ' '.join(x for x in x.split() if  not x.isdigit()))
    data[c] = data[c].apply(lambda x:' '.join(x for x in x.split() if not x in stop))
    data[c] = data[c].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
  return data


In [120]:
data_train = dataPreProc(data_train)
data_train.head(3)

,question,answer_text,op0,op1,op2
0,meal served,room p,outside room p,dining room p,dining room p
1,inferred passage,local government deal problem lacking money mean,tragedy occur,relevant department state council take respons...,currently
3,inferred passage,writer willing use idiom,idiom important part language,nonnative speaker learn idiom,way master idiom


In [122]:
data_test = dataPreProc(data_test)
data_test.head(3)

,question,answer_text,op0,op1,op2
0,meal served,room p,outside room p,dining room p,dining room p
1,inferred passage,local government deal problem lacking money mean,tragedy occur,relevant department state council take respons...,currently
3,inferred passage,writer willing use idiom,idiom important part language,nonnative speaker learn idiom,way master idiom


In [0]:
x_train = data_train.iloc[:,0:2]
y_train  = data_train.iloc[:,2:5]
x_test = data_test.iloc[:,0:2] 
y_test  = data_test.iloc[:,2:5]

In [0]:
from nltk.corpus import stopwords
import string
from textblob import Word
stop = stopwords.words('english')

In [0]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer()
total_x = x_train + x_test
total_y = y_train + y_test

In [171]:
len(total_y)

13595

In [144]:
for c in total_x:
  tokenizer.fit_on_texts(total_x[c])
for c in total_y:
  tokenizer.fit_on_texts(total_y[c])
vocab_size = len(tokenizer.word_index)+1
vocab_size

67586

In [0]:
#tokenize
x_train_tokens =pd.DataFrame()
y_train_tokens =pd.DataFrame()
x_test_tokens = pd.DataFrame()
y_test_tokens = pd.DataFrame()
for c in total_x:
  x_train_tokens[c] = tokenizer.texts_to_sequences(x_train[c])
  x_test_tokens[c] = tokenizer.texts_to_sequences(x_test[c])
for c in total_y:
  y_train_tokens[c] = tokenizer.texts_to_sequences(y_train[c])
  y_test_tokens[c] = tokenizer.texts_to_sequences(y_test[c])

In [149]:
x_train_tokens.head()

,question,answer_text
0,"[619, 1656]","[406, 2389]"
1,"[204, 2]","[394, 172, 336, 64, 2586, 38, 136]"
2,"[204, 2]","[41, 995, 30, 3469]"
3,"[41, 64, 1157, 2679, 10, 35]","[2280, 74, 36]"
4,"[811, 4168, 198, 816, 573, 19096]","[2110, 7981, 16, 1901]"


In [150]:
y_train_tokens.head()

,op0,op1,op2
0,"[703, 406, 2389]","[2154, 406, 2389]","[2154, 406, 2389]"
1,"[3441, 2637]","[4396, 1390, 222, 2607, 44, 1308, 1018]",[4922]
2,"[3469, 65, 102, 85]","[46181, 984, 22, 3469]","[31, 1114, 3469]"
3,"[3, 17, 13, 10]","[3, 18, 572, 92, 1451]","[3, 17, 13, 83]"
4,"[2110, 7981, 700]","[2110, 8973, 748, 833, 4048]","[2110, 8973, 16]"


In [183]:
#padding
padX = 0
for c in total_x:
  padX = max([len(s.split()) for s in total_x[c]])
padY = 0
for c in total_y:
  padY = max([len(s.split()) for s in total_y[c]])
maxPad = padX if padX >= padY else padY
x_train_pad =pd.DataFrame()
y_train_pad =pd.DataFrame()
x_test_pad = pd.DataFrame()
y_test_pad = pd.DataFrame()

x_train_pad['question'] = pad_sequences(x_train_tokens['question'],maxlen=maxPad,padding='post')
x_test_pad['question'] = pad_sequences(x_test_tokens['question'],maxlen=maxPad,padding='post')
# for c in total_y:
#   y_train_pad = pad_sequences(y_train_tokens[c],maxlen=maxPad,padding='post')
#   y_test_pad= pad_sequences(y_test_tokens[c],maxlen=maxPad,padding='post')

Exception: ignored

In [180]:
x_train_pad

array([[ 406, 2389,    0, ...,    0,    0,    0],
       [ 394,  172,  336, ...,    0,    0,    0],
       [  41,  995,   30, ...,    0,    0,    0],
       ...,
       [ 415, 1332, 1555, ...,    0,    0,    0],
       [ 123,  603,    9, ...,    0,    0,    0],
       [ 121, 2037, 4269, ...,    0,    0,    0]], dtype=int32)